In [ ]:
import os
import numpy as np
from numpy.linalg import norm
from skimage.util.shape import *
from operator import mul, sub
from math import floor, sqrt, log10
import sys
from scipy.sparse.linalg import svds
from scipy.stats import chi2
from skimage.util import pad
import timeit
import os
os.sys.path
from skimage import io
import numpy as np
import PIL.Image
from PIL import Image, ImageFile  # pour convertir une image en niveaux de gris
import os


In [ ]:
def standardization(m):
  n,p=m.shape
  means = np.zeros(p)
  stds = np.zeros(p)
  for i in range(p):
    sum = 0
    mean = 0
    std = 0
    for j in range(n):
      sum += m[j][i]
    mean = sum/n
    sum = 0
    for j in range(n):
      sum += (m[j][i]-mean)**2
    std = np.sqrt(1/n*sum)
    sum = 0
    for j in range(n):
      m[j][i] = (m[j][i] -mean)/std
      means[i]=mean
      stds[i]=std
  return m,means,stds

In [ ]:
def OMP(X,D,kmax,eps):
    R = X #création du résiduel
    alpha = np.zeros(len(D[0])) # création de alpha
    support = []
    k = 0
    m =[0 for i in range (len(D[0]))]
    while np.linalg.norm(R) > eps and k < kmax : # critere d'arret
        #print(R)
        for j in range (len(D[0])):
            dj = np.array(D[:,[j]])
            m[j] = abs((np.vdot(dj,R)) / (np.linalg.norm(dj)))
        mk=np.argmax(m)
        support.append(mk)
       # print(len(support),"test")
        #print(support)
        phi = np.array(D[:,support])
       # print(phi)
        alpha[support] = (np.linalg.pinv(phi)).dot(X)
        R = X-D.dot(alpha)
        k=k+1
    return (alpha)

In [ ]:
def Initialisation_dictionnaire(X,K):
  (N,l) = X.shape
  D = np.zeros((N,K))
  for k in range(K):
    normDcolonne = norm(X[:, k])
    for i in range(N):
      D[i, k] = X[i,k] / normDcolonne
  return D
DicoInitial=Initialisation_dictionnaire(DicoInitial,500)


def realisationDeOMP(X,K,Nmax,eps):
  (N,l)=X.shape
  alpha=np.zeros((K,len(X[0])))
  #Nombre d'iteration maximal à effectuer pour OMP
  MAX_bruiteeITR=np.round(K/10);
  D0=Initialisation_dictionnaire(X,K)

  for i in range(1,l):
    alpha[:,j]=OMP(X[:,j],D0,10,eps)
  return alpha



def Lambda(X,D):
  (N, ell) = X.shape
  (N, K) = D.shape
  Lambda = np.zeros((K, ell))
  for j in range(ell):
    x = X[:, j]
    #x = x.reshape((N, 1))
    alpha = OMP(x,D,50,eps)
    for i in range(K):
      Lambda[i,j] = alpha[i]

  return Lambda


  # Définition d'une fonction pour déterminer le support
def support(A_k):
  w = []
  for k in range(len(A_k)):
    if A_k[k] != 0:
     w.append(k)
  return w

def Omega(support,l):
  card = len(support)
  omega = np.zeros((l, card))
  for a in range(card):
    omega[support[a], a] = 1
  return omega


def k_svd(D,Lambda,X):
  (N, l) = X.shape
  (N, K) = D.shape
  for k in range(K):
    w_k = support(Lambda[k, :])
    card = len(w_k)
    if card != 0:
      E_k = X - np.dot(D, Lambda) + np.dot(D[:, k].reshape((N, 1)), Lambda[k, :].reshape((1, l)))
      omega = np.zeros((l, card))
  for a in range(card):
    omega[w_k[a], a] = 1
    E_kr = np.dot(E_k, omega)
    U, S, V = np.linalg.svd(E_kr)
  for i in range(N):
    D[i, k] = U[i, 0]
  for j in range(len(w_k)):
   Lambda[k, w_k[j]] = S[0] * V[0, j]
  return D, Lambda

def D_0(X,K):
    (N,ell) = X.shape
    D = np.zeros((N, K))
    for k in range(K):
        normXcolonne = norm(X[:, k])
        for i in range(N):
            D[i, k] = X[i,k] / normXcolonne
    return D

def dictionary_learning(X, K, L):
  D = D_0(X, K)
  D=Initialisation_dictionnaire(X,K)
  for l in range(L):
    A = Lambda(X,D)
    [D,A]= k_svd(D,A,X)

  return D,A

DicoInitial.shape


(100, 500)

In [ ]:
K=1000
L=2
Dlearning,A=dictionary_learning(DicoInitial,314,1)##K=100



In [ ]:
def inverseTransform(M,means,stds):
  n,p = M.shape
  for i in range(p):
    for j in range(n):
      M[j,i]=M[j][i]*stds[i]+means[i]
  return M